In [ ]:
# =============================================
# 00_build_light_index.ipynb - Step 1
# Build a clean index of available images （merge eval/item/bbox）
# 构建干净的图片索引文件（整合 eval/item/bbox）
# =============================================

import os
import pandas as pd

# ----- Path Configuration (路径配置) -----
DATA_DIR = '../data'                       # Root data folder （数据根目录）
IMG_ROOT = os.path.join(DATA_DIR, 'img')   # Image folder （图片文件夹）
OUT_CSV  = os.path.join(DATA_DIR, 'images_clean.csv')  # Output path （输出路径）

# Input raw text files (原始文件路径)
fp_eval  = os.path.join(DATA_DIR, 'list_eval_partition.txt')
fp_item  = os.path.join(DATA_DIR, 'list_item_inshop.txt')
fp_bbox  = os.path.join(DATA_DIR, 'list_bbox_inshop.txt')

print("[INFO] DATA_DIR =", os.path.abspath(DATA_DIR))
print("[INFO] IMG_ROOT =", os.path.abspath(IMG_ROOT))

# Quick check that all input files exist (检查文件是否存在)
for p in [fp_eval, fp_item, fp_bbox]:
    print(f"[CHECK] {os.path.basename(p)} ->", "✅ OK" if os.path.exists(p) else "❌ NOT FOUND")


[INFO] DATA_DIR = d:\Icey\tcd\notebooks\dMining\Group Project\workflow\data
[INFO] IMG_ROOT = d:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\img
[CHECK] list_eval_partition.txt -> ✅ OK
[CHECK] list_item_inshop.txt -> ✅ OK
[CHECK] list_bbox_inshop.txt -> ✅ OK


In [14]:
def norm_img_rel(p: str) -> str:
    """
    Normalize image paths to avoid duplicates like 'img/img/...'
    标准化图片路径，去掉多余的 img/img 层级
    Example (示例):
        'data/img/img/WOMEN/Dresses/x.jpg' -> 'WOMEN/Dresses/x.jpg'
    """
    p = str(p).replace('\\','/').strip().replace('img/img/', 'img/')
    if p.startswith('img/'):
        p = p[len('img/'):]
    return p


In [ ]:
# 00_build_light_index.ipynb - Cell 3 (robust reader for your 3-column eval)
import re

def read_eval(fp):
    """
    Robust reader for list_eval_partition.txt
    支持以下几种格式 support the following formats：
      A) image_name  item_id  train|query|gallery   <-- your format ✅
      B) image_name  train|query|gallery
      C) image_name  其它字段 ... 再跟 train|query|gallery
    产出列：image_path, eval_split, item_id_eval(如果有)
    """
    rows = []
    with open(fp, 'r', encoding='utf-8') as f:
        for line in f:
            s = line.strip().split()
            if not s:
                continue
            # 跳过表头/注释行
            if any(k in s[0].lower() for k in ['image', 'im_name', '#', 'file', 'image_name']):
                continue

            img = s[0]
            rest = [t.strip() for t in s[1:]]
            split = None
            item_id_eval = None

            # 情况 A：后三列之一含有 train/query/gallery，同时第二列是 item_id
            # e.g. img path | id_xxx | train
            for t in rest[::-1]:  # 从右向左找 split
                tl = t.lower()
                if tl in ('train','query','gallery'):
                    split = tl
                    break
            # 尝试把第二列当作 item_id（若存在且像 id_XXXX）
            if len(rest) >= 1 and rest[0].startswith('id_'):
                item_id_eval = rest[0]

            # 情况 B：两列格式：img | train
            if split is None and len(rest) == 1 and rest[0].lower() in ('train','query','gallery'):
                split = rest[0].lower()

            # 若仍无法判断，跳过该行（也可设置默认 'train'）
            if split is None:
                # print("skip unknown eval line:", s)  # 调试用
                continue

            rows.append((norm_img_rel(img), split, item_id_eval))

    df = pd.DataFrame(rows, columns=['image_path','eval_split','item_id_eval']).drop_duplicates()
    return df

def read_item(fp):
    """标准两列：image_path item_id"""
    rows=[]
    with open(fp,'r',encoding='utf-8') as f:
        for line in f:
            s=line.strip().split()
            if not s: 
                continue
            if any(k in s[0].lower() for k in ['image','im_name','#','file']):
                continue
            if len(s)>=2:
                rows.append((norm_img_rel(s[0]), s[1]))
    return pd.DataFrame(rows, columns=['image_path','item_id']).dropna().drop_duplicates()

def read_bbox(fp):
    """最后4个数字当作 x1,y1,x2,y2"""
    rows=[]
    with open(fp,'r',encoding='utf-8') as f:
        for line in f:
            s=line.strip().split()
            if not s: 
                continue
            if any(k in s[0].lower() for k in ['image','im_name','#','file']):
                continue
            nums=[t for t in s[1:] if t.replace('-','').isdigit()]
            if len(nums)>=4:
                x1,y1,x2,y2=map(int, nums[-4:])
                rows.append((norm_img_rel(s[0]), x1,y1,x2,y2))
    return pd.DataFrame(rows, columns=['image_path','x1','y1','x2','y2']).drop_duplicates()

#  read files （实际读取）
df_eval = read_eval(fp_eval)
df_item = read_item(fp_item)
df_bbox = read_bbox(fp_bbox)

print(f"[INFO] eval rows = {len(df_eval)}  cols: {list(df_eval.columns)}")
print(f"[INFO] item rows = {len(df_item)}  cols: {list(df_item.columns)}")
print(f"[INFO] bbox rows = {len(df_bbox)}  cols: {list(df_bbox.columns)}")

display(df_eval.head(3))
display(df_item.head(3))
display(df_bbox.head(3))


[INFO] eval rows = 52712  cols: ['image_path', 'eval_split', 'item_id_eval']
[INFO] item rows = 0  cols: ['image_path', 'item_id']
[INFO] bbox rows = 52712  cols: ['image_path', 'x1', 'y1', 'x2', 'y2']


,image_path,eval_split,item_id_eval
0,WOMEN/Dresses/id_00000002/02_1_front.jpg,train,id_00000002
1,WOMEN/Dresses/id_00000002/02_2_side.jpg,train,id_00000002
2,WOMEN/Dresses/id_00000002/02_4_full.jpg,train,id_00000002


,image_path,item_id


,image_path,x1,y1,x2,y2
0,WOMEN/Blouses_Shirts/id_00000001/02_1_front.jpg,50,49,208,235
1,WOMEN/Blouses_Shirts/id_00000001/02_2_side.jpg,119,48,136,234
2,WOMEN/Blouses_Shirts/id_00000001/02_3_back.jpg,50,42,213,240


In [ ]:
# 00_build_light_index.ipynb - Cell 4 (merge with item_id preference)

# First, merge eval with item, and if eval contains item_id_eval, use it preferentially
df_ei = df_eval.merge(df_item, on='image_path', how='left')  # item_id可能为空
# choose priority: item_id > item_id_eval (选用优先级：item_id > item_id_eval)
df_ei['item_id_final'] = df_ei['item_id_eval'].fillna(df_ei['item_id'])
df_ei.drop(columns=['item_id'], inplace=True, errors='ignore')  # avoid name conflict (避免重名困扰)
df_ei.rename(columns={'item_id_final':'item_id'}, inplace=True)

# Then merge with bbox (再与 bbox 合并)
df = df_ei.merge(df_bbox, on='image_path', how='inner').drop_duplicates(subset=['image_path'])

# If eval_split is not in the column (very rare case), add it one more time  （若 eval_split 不在列中（极少数情况），再补一次）
if 'eval_split' not in df.columns:
    print("[FIX] add eval_split back from df_eval ...")
    df = df.merge(df_eval[['image_path','eval_split']], on='image_path', how='left')

#  Picture Existence Check （图片存在性检查）
def exists(rel):
    return os.path.exists(os.path.join(IMG_ROOT, rel))

if 'exists' not in df.columns:
    df['exists'] = df['image_path'].apply(exists)

missing = (~df['exists']).sum()
if missing > 0:
    print(f"[WARN] Missing files: {missing} (show first 5)")
    display(df.loc[~df['exists'], ['image_path']].head(5))

df = df[df['exists']].copy()
df.drop(columns=['exists'], inplace=True, errors='ignore')
df.reset_index(drop=True, inplace=True)

print("✅ Clean rows:", len(df))
print("📄 Columns:", list(df.columns))

# Distribution Check （分布检查）
if 'eval_split' in df.columns:
    try:
        display(df['eval_split'].value_counts())
    except:
        print(df['eval_split'].value_counts())
else:
    print("❌ Still missing eval_split. Please show me the exact first 10 lines of list_eval_partition.txt")

# Preview
display(df.head(5))


✅ Clean rows: 52712
📄 Columns: ['image_path', 'eval_split', 'item_id_eval', 'item_id', 'x1', 'y1', 'x2', 'y2']


eval_split
train      25882
query      14218
gallery    12612
Name: count, dtype: int64

,image_path,eval_split,item_id_eval,item_id,x1,y1,x2,y2
0,WOMEN/Dresses/id_00000002/02_1_front.jpg,train,id_00000002,id_00000002,65,45,233,252
1,WOMEN/Dresses/id_00000002/02_2_side.jpg,train,id_00000002,id_00000002,112,41,168,247
2,WOMEN/Dresses/id_00000002/02_4_full.jpg,train,id_00000002,id_00000002,89,34,169,167
3,WOMEN/Dresses/id_00000002/02_7_additional.jpg,train,id_00000002,id_00000002,73,40,194,251
4,WOMEN/Skirts/id_00000003/02_1_front.jpg,train,id_00000003,id_00000003,51,122,160,210


In [17]:
# === Save merged dataset for later steps ===
import os

# 路径指向你的 data 目录
OUT_CSV = r"D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\images_clean.csv"

df.to_csv(OUT_CSV, index=False)
print("✅ Saved merged CSV to:", OUT_CSV)
print("Rows:", len(df))
print("Columns:", list(df.columns))


✅ Saved merged CSV to: D:\Icey\tcd\notebooks\dMining\Group Project\workflow\data\images_clean.csv
Rows: 52712
Columns: ['image_path', 'eval_split', 'item_id_eval', 'item_id', 'x1', 'y1', 'x2', 'y2']


In [4]:
import pandas as pd
import os

# 你的 keys.csv 路径
CSV_PATH = os.path.join("D:/Icey/tcd/notebooks/dMining/Group Project/workflow/data/keys.csv")

# 读取 keys.csv
df = pd.read_csv(CSV_PATH)
print(f"✅ Loaded keys.csv, shape = {df.shape}")

# 检查 image_path 列是否存在
if 'image_path' not in df.columns:
    raise ValueError("❌ keys.csv 中没有 image_path 列，请检查文件内容")

# 提取 gender & category
def parse_gender_category(p):
    if pd.isna(p):
        return pd.Series(["UNK", "UNK"])
    p = str(p).replace("\\", "/").lstrip("/")
    parts = p.split("/")
    # 去掉 img 前缀（若存在）
    if len(parts) > 0 and parts[0].lower() == 'img':
        parts = parts[1:]
    gender = parts[0] if len(parts) > 0 else "UNK"
    category = parts[1] if len(parts) > 1 else "UNK"
    return pd.Series([gender, category])

df[['gender', 'category']] = df['image_path'].apply(parse_gender_category)

# 显示前几行确认
print("✅ Extracted columns:")
print(df[['image_path', 'gender', 'category']].head())

# 保存回原文件（覆盖）
df.to_csv(CSV_PATH, index=False)
print(f"✅ Updated keys.csv saved successfully: {CSV_PATH}")


✅ Loaded keys.csv, shape = (26830, 4)
✅ Extracted columns:
                                        image_path gender        category
0  WOMEN/Blouses_Shirts/id_00000001/02_1_front.jpg  WOMEN  Blouses_Shirts
1   WOMEN/Blouses_Shirts/id_00000001/02_2_side.jpg  WOMEN  Blouses_Shirts
2   WOMEN/Blouses_Shirts/id_00000001/02_3_back.jpg  WOMEN  Blouses_Shirts
3   WOMEN/Blouses_Shirts/id_00000001/02_4_full.jpg  WOMEN  Blouses_Shirts
4      WOMEN/Tees_Tanks/id_00000007/01_1_front.jpg  WOMEN      Tees_Tanks
✅ Updated keys.csv saved successfully: D:/Icey/tcd/notebooks/dMining/Group Project/workflow/data/keys.csv
